In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train=pd.read_csv("0000000000002747_training_twitter_x_y_train.csv")
test=pd.read_csv("0000000000002747_test_twitter_x_test.csv")

In [3]:
# Dropping Useless Columns

train.drop('tweet_id', inplace = True, axis = 1)
train.drop('airline_sentiment_gold', inplace = True, axis = 1)
train.drop('negativereason_gold', inplace = True, axis = 1)
train.drop('tweet_coord', inplace = True, axis = 1)
train.drop('tweet_created',inplace = True, axis = 1)

In [4]:

mood_count = train['airline_sentiment'].value_counts()
mood_count

negative    6851
neutral     2327
positive    1802
Name: airline_sentiment, dtype: int64

In [5]:
train['airline'].value_counts()

United            2928
US Airways        2152
American          2078
Southwest         1817
Delta             1639
Virgin America     366
Name: airline, dtype: int64

In [6]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['airline_sentiment'] = le.fit_transform(train['airline_sentiment'])
train.head(2)

,airline_sentiment,airline,name,retweet_count,text,tweet_location,user_timezone
0,0,Southwest,ColeyGirouard,0,"@SouthwestAir I am scheduled for the morning, ...",Washington D.C.,Atlantic Time (Canada)
1,2,Southwest,WalterFaddoul,0,@SouthwestAir seeing your workers time in and ...,"Indianapolis, Indiana; USA",Central Time (US & Canada)


In [7]:
import re
import nltk
from nltk.corpus import stopwords

In [8]:
def tweet_to_words(raw_tweet):
    letters_only = re.sub("[^a-zA-Z]", " ",raw_tweet) 
    words = letters_only.lower().split()                             
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops] 
    return( " ".join( meaningful_words ))


In [9]:
train['clean_tweet'] = train['text'].apply(lambda x: tweet_to_words(x))

test['clean_tweet'] = test['text'].apply(lambda x: tweet_to_words(x))

In [10]:
train_clean_tweet=[]
for tweet in train['clean_tweet']:
    train_clean_tweet.append(tweet)
    
test_clean_tweet=[]
for tweet in test['clean_tweet']:
    test_clean_tweet.append(tweet)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(analyzer = "word")
train_features= v.fit_transform(train_clean_tweet)
test_features= v.transform(test_clean_tweet)

In [12]:
from sklearn.naive_bayes import MultinomialNB

In [13]:

nb = MultinomialNB()

In [14]:
nb.fit(train_features,train['airline_sentiment'])
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy = accuracy_score(nb.predict(train_features),train['airline_sentiment'])
print(accuracy)

0.8437158469945355


In [15]:
test_pred = nb.predict(test_features)
test_pred = le.inverse_transform(test_pred)
print(test_pred)
np.savetxt("test_pred.csv",test_pred,fmt='%s')

['negative' 'negative' 'negative' ... 'neutral' 'positive' 'neutral']


In [16]:
from sklearn.ensemble import RandomForestClassifier

In [17]:

clf = RandomForestClassifier(n_estimators = 200)

In [18]:
clf.fit(train_features,train['airline_sentiment'])

RandomForestClassifier(n_estimators=200)

In [19]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy = accuracy_score(clf.predict(train_features),train['airline_sentiment'])
print(accuracy)

0.995719489981785


In [20]:
test_pred2 = clf.predict(test_features)

In [21]:
test_pred2 = le.inverse_transform(test_pred2)

In [22]:
np.savetxt("test_pred2.csv",test_pred2,fmt='%s')